# SPINE PROBLEMS

## 0 IMPORT LIBRARIES

verify pytorch

In [1]:
import cv2
import numpy as np
import polars as pl
import gc
import importlib
import os
import random
import shutil
import subprocess
from typing import Any, Callable
import pydicom
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from tqdm.notebook import tqdm

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('Running on the GPU')
else:
    device = torch.device('cpu')
    print('Running on the CPU')

Running on the CPU


### 1 GLOBAL SEATINGS

In [3]:
WORKDIR_PATH = "" # "../"
IMAGE_DIM = 256
NO_CACHE = True
CACHE_DIR = WORKDIR_PATH + ".cache/"
DATASET_DIR = WORKDIR_PATH + "dataset/"
ANOTTATIONS_DIR = DATASET_DIR + "annotations/"
TRAIN_DIR = os.path.join(DATASET_DIR, "train_images/")
print(TRAIN_DIR)
LOAD_SESSION = False
RESOURCES = "resources/"

dataset/train_images/


## 2 Preprocessing

### 2.2 Load Dataset

In [4]:
lazy_dataset = any
if os.path.exists(CACHE_DIR + "preprocesed_dataset.parquet"):
    print("cache")
    lazy_dataset = pl.scan_parquet(CACHE_DIR + 'preprocesed_dataset.parquet')
    LENGTH = lazy_dataset.select(pl.len()).collect().item()
    NO_CACHE = False
else:
    print("from zero")
    lazy_dataset = pl.scan_csv(ANOTTATIONS_DIR + 'train.csv')
    # lazy_dataset = pl.scan_csv(ANOTTATIONS_DIR + 'p.csv')
    LENGTH = lazy_dataset.select(pl.len()).collect().item()

cache


### 2.3 Filter dataset 

In [5]:
if NO_CACHE:
    lazy_dataset = (
        lazy_dataset
        .select(['image_id', 'lesion_type'])
    )

In [6]:
if NO_CACHE:
    lazy_dataset = (
        lazy_dataset.group_by("image_id").agg(
        pl.col("lesion_type"))
    )
    LENGTH = lazy_dataset.select(pl.len()).collect().item()

### 2.4 Get image path and add to the dataset

In [7]:
if NO_CACHE:
    lazy_dataset = (
        lazy_dataset
        .with_columns((pl.lit(TRAIN_DIR) + (pl.col("image_id")+pl.lit('.dicom'))).alias("image_path"))
        .drop('image_id')
    )

### 2.6 Preprocess the data

In [8]:
from utils import process_data
from utils import visualice

if NO_CACHE:
    lazy_dataset = (
        lazy_dataset
        .with_columns(
            pl.col("image_path")
            .map_elements(
                function=process_data.preprocess,
                return_dtype=pl.List(pl.List(pl.Float32))
                )
            .alias("image")
        )
        .drop("image_path")
    )
visualice.visualice_lazyframe(lazy_dataset)

lesion_type,image
str,list[list[f32]]
"""Osteophytes""","[[0.0, 0.0, … 0.0], [0.0, 0.0, … 0.0], … [0.0, 0.0, … 0.0]]"
"""Osteophytes""","[[0.0, 0.0, … 0.0], [0.0, 0.0, … 0.0], … [0.0, 0.0, … 0.0]]"
"""Osteophytes""","[[0.0, 0.0, … 0.0], [0.0, 0.0, … 0.0], … [0.0, 0.0, … 0.0]]"
"""Osteophytes""","[[0.0, 0.0, … 0.0], [0.0, 0.0, … 0.0], … [0.0, 0.0, … 0.0]]"
"""Osteophytes""","[[0.442811, 0.441742, … 0.187867], [0.450027, 0.445216, … 0.179583], … [0.036879, 0.036879, … 0.461251]]"


### 2.7 Save the preprocessed dataset

In [9]:
from utils import create_parquet
if NO_CACHE:
    create_parquet.process_lazy_images(lazy_dataset,total_rows=LENGTH, chunk_size=50, output_path=CACHE_DIR)

In [10]:
import polars as pl
import torch
from torch.utils.data import Dataset

class LazyFrameDataset(Dataset):
    def __init__(self, lazy_frame: pl.LazyFrame, target_column:str='image', dataset_length:int = 0):
        self.length = dataset_length
        self.target_column = target_column
        self.lf = lazy_frame
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        row = (
            self.lf
            .select([self.target_column])
            .slice(idx, 1)
            .collect()
            .row(0)
        )
        image = torch.tensor(row[0], dtype=torch.float32)
        image = image.unsqueeze(0)
        return image

## 3 AUTOENCODER

### 3.1 Covolutional Block

In [11]:

class ConvolutionBlock(nn.Module):
    def __init__(self, in_channels, out_channels, dropout_rate:float, kernel_size:int, activation):
        super().__init__()
        padding = kernel_size // 2
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding),
            nn.BatchNorm2d(out_channels),
            activation,
            nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=padding),
            nn.BatchNorm2d(out_channels),
            activation,
            nn.Dropout2d(dropout_rate)
        )
    def forward(self, x):
        return self.block(x)

### 3.2 Encoder Block

In [12]:
class Encoder(nn.Module):
    def __init__(self, in_channels:int=1, base_filters:int=64):
        super().__init__()

        self.in_channels = in_channels
        self.base_filters = base_filters

        self.pool = nn.MaxPool2d((2, 2))

        self.level1 = ConvolutionBlock(self.in_channels, self.base_filters,dropout_rate=0.2, kernel_size=3, activation=nn.ReLU())
        self.level2 = ConvolutionBlock(self.base_filters, self.base_filters * 2,dropout_rate=0.2, kernel_size=3, activation=nn.ReLU())
        self.level3 = ConvolutionBlock(self.base_filters * 2, self.base_filters * 4,dropout_rate=0.2, kernel_size=3, activation=nn.ReLU())
        
        self.bottleneck = ConvolutionBlock(self.base_filters * 4, self.base_filters * 8,dropout_rate=0.2, kernel_size=3, activation=nn.ReLU())
        
    def forward(self, x):

        skip_connections = []

        level1_features = self.level1(x)
        skip_connections.append(level1_features)
        level1_pooled = self.pool(level1_features)

        level2_features = self.level2(level1_pooled)
        skip_connections.append(level2_features)
        level2_pooled = self.pool(level2_features)

        level3_features = self.level3(level2_pooled)
        skip_connections.append(level3_features)
        level3_pooled = self.pool(level3_features)

        bottleneck_features = self.bottleneck(level3_pooled)
      
        return {
            'encoded_image': bottleneck_features,
            'skip_connections': skip_connections
        }

### 3.3 Decoder


In [13]:
class Decoder(nn.Module):
    def __init__(self, in_channels:int=1, base_filters:int=64):
        super().__init__()
        
        self.in_channels = in_channels
        self.base_filters = base_filters * 8
        
        self.upconv1 = nn.ConvTranspose2d(self.base_filters//1, self.base_filters // 2, kernel_size=2, stride=2)
        self.upconv2 = nn.ConvTranspose2d(self.base_filters // 2, self.base_filters // 4, kernel_size=2, stride=2)
        self.upconv3 = nn.ConvTranspose2d(self.base_filters // 4, self.base_filters // 8, kernel_size=2, stride=2)
        
        self.conv1 = ConvolutionBlock(self.base_filters, self.base_filters // 2, dropout_rate=0.2, kernel_size=3, activation=nn.ReLU())
        self.conv2 = ConvolutionBlock(self.base_filters // 2, self.base_filters // 4, dropout_rate=0.2, kernel_size=3, activation=nn.ReLU())
        self.conv3 = ConvolutionBlock(self.base_filters // 4, self.base_filters // 8, dropout_rate=0.2, kernel_size=3, activation=nn.ReLU())
        self.output_conv = nn.Conv2d(self.base_filters // 8, in_channels, kernel_size=1)
        
    def forward(self, encoder_output):

        features = encoder_output['encoded_image']
        skip_connections = encoder_output['skip_connections']
        up1 = self.upconv1(features)
        up1 = torch.cat([up1, skip_connections[2]], dim=1)
        up1 = self.conv1(up1)

        up2 = self.upconv2(up1)
        up2 = torch.cat([up2, skip_connections[1]], dim=1)
        up2 = self.conv2(up2)

        up3 = self.upconv3(up2)
        up3 = torch.cat([up3, skip_connections[0]], dim=1)
        up3 = self.conv3(up3)

        output = torch.sigmoid(self.output_conv(up3))
        return output

### 3.4 Autoencoder

In [14]:
class Autoencoder(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        
        self.encoder = encoder
        self.encoder.to(device)
        
        self.decoder = decoder
        self.decoder.to(device)
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

## 4 Model

Split the dataset into train and test

In [16]:
train_dataset, validation_dataset = torch.utils.data.random_split(LazyFrameDataset(lazy_frame=lazy_dataset, target_column='image', dataset_length=LENGTH), [0.8, 0.2])

Create the model

In [17]:
from utils.training import CNN
model = CNN.ConvolutionalAutoencoder(autoencoder=Autoencoder(Encoder(), Decoder()),training_set=train_dataset, batch_size=1, device=device)

Train the model

In [18]:
a, b = model.train(num_epochs=50)

Training process:   0%|          | 0/15649 [00:00<?, ?it/s]

KeyboardInterrupt: 

### 4.1 Training function

In [14]:
def train_autoencoder(data: LazyFrameDataset, num_epochs: int = 50, batch_size: int = 8, learning_rate: float = 1e-3, device: str = None):
    # Set up device
    if device is None:
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Initialize models and move to device
    encoder = Encoder().to(device)
    model = Autoencoder(encoder).to(device)

    dataloader = DataLoader(
        data,
        batch_size=batch_size,
        shuffle=True,
        pin_memory=True
    )
    # Loss function
    criterion = nn.MSELoss()
   
    # Optimizer
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        total_loss = 0.0
        i = 1
        print(f"Epoch {epoch + 1}")
        # for batch_index, batch in enumerate(dataloader):
        #     batch = batch.to(device)
        #     print(f"{batch_index}/{len(dataloader)}", batch.shape)
        for batch_index, batch in enumerate(dataloader):
            batch = batch.to(device)
            optimizer.zero_grad()
            # No need to call .cuda() here since model is already on the correct device
            reconstructed = model(batch)
            
            loss = criterion(reconstructed, batch)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            print(f"\tProgress: ({i}/{len(dataloader)})", end='\r', flush=True)
            i += 1
            if batch_index % 10 == 0:
                print(f"Batch {batch_index}/{len(dataloader)}, "
                          f"Loss: {loss.item():.4f}, "
                          f"Batch shape: {batch.shape}", end='\r', flush=True)
            
        avg_loss = total_loss / len(data)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')
    
    return model, encoder

### 4.2 Lazyframe slpiter function

In [15]:
def split_lazyframe(
    lf: pl.LazyFrame, dataset_length:int=0,train_fraction: float = 0.75
) -> tuple[pl.LazyFrame, pl.LazyFrame]:
    """Split polars dataframe into two sets.
    Args:
        df (pl.DataFrame): Dataframe to split
        train_fraction (float, optional): Fraction that goes to train. Defaults to 0.75.
    Returns:
        Tuple[pl.DataFrame, pl.DataFrame]: Tuple of train and test dataframes
    """
    data_slpit = int(dataset_length*train_fraction) 
    print(dataset_length, data_slpit)
    df_train = lf.slice(0, data_slpit)
    df_test = lf.slice(data_slpit+1, dataset_length-data_slpit)
    return df_train, df_test

### 4.3 Training

In [ ]:
# train_lf, test_lf = split_lazyframe(lazy_dataset, dataset_length=lazy_dataset.select(pl.len()).collect().item())
print(LENGTH)
trained_model, trained_encoder = train_autoencoder(LazyFrameDataset(lazy_dataset,'image', LENGTH),num_epochs=50,batch_size=1)

# Optional: Save the trained encoder for later use
torch.save(trained_encoder.state_dict(), 'trained_encoder.pth')

19746
Using device: cuda:0
Epoch 1


OutOfMemoryError: CUDA out of memory. Tried to allocate 15.94 GiB. GPU 0 has a total capacity of 12.00 GiB of which 0 bytes is free. Process 887339 has 17179869184.00 GiB memory in use. Of the allocated memory 16.22 GiB is allocated by PyTorch, and 14.59 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## AUTOENCODER

<img src="resources/Structure-of-autoencoder-for-feature-extraction.png" alt="Autoencoder PHoto" width="600px" height="300px"/>

We use an autoencoder for dimensionality reduction, in order to 